In [42]:
import pandas as pd
import pyodbc

# Initialize connection parameters

In [43]:
DB = {'servername': 'MSI',
      'source': 'AdventureWorks2019',
      'destination':'DW',
      'user':'user1',
      'pass':'123456'}

# Create connection

In [44]:
#SourceDB
sourceConn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + DB['servername'] + ';DATABASE=' + DB['source'] +';UID='+ DB['user']+';PWD='+DB['pass']+ ';Trusted_Connection=yes')

#DestinationDB
destConn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + DB['servername'] + ';DATABASE='+DB['destination'] +';UID='+ DB['user']+';PWD='+DB['pass']+ ';Trusted_Connection=yes')
cursor=destConn.cursor()

In [45]:
df = pd.read_sql('SELECT * FROM Purchasing.[PurchaseOrderHeader]',sourceConn)
df['ModifiedDate'] =pd.to_datetime(df['ModifiedDate']).dt.date
date=df['ModifiedDate'].drop_duplicates().reset_index(drop=True)
print(date)

0      2011-04-25
1      2011-05-09
2      2011-12-23
3      2011-12-24
4      2012-01-17
          ...    
292    2014-08-09
293    2014-08-10
294    2014-08-11
295    2014-08-12
296    2015-08-12
Name: ModifiedDate, Length: 297, dtype: object


c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [46]:
# for index, row in date.iterrows():
#       cursor.execute('''INSERT INTO dbo.DateDim (date) values(?)''', row.ModifiedDate)
#       cursor.commit()
# datedim2 = pd.read_sql('SELECT * FROM dbo.DateDim',destConn)
# print(datedim2)


# Create DIM tables

In [47]:
cursor.execute('''CREATE TABLE DimDate (
    FullDate date  primary key,
    DayOfWeek varchar(10)
)''')
cursor.commit()


In [48]:
cursor.execute('''CREATE TABLE DimShipMethod (
    ShipMethodID int  primary key,
    Name varchar(50)
)''')

cursor.execute('''CREATE TABLE DimProduct (
    ProductID int primary key,
    Name varchar(50),
    Color varchar(50),
    Size varchar(50),
    Class varchar(50),
    Style varchar(50)
)''')

cursor.execute('''CREATE TABLE DimEmployee (
    EmployeeID int primary key,
    Name varchar(50)
)''')

cursor.execute('''CREATE TABLE DimVendor (
    VendorID int primary key,
    Name varchar(50),
    CreditRating varchar(20)
)''')

cursor.execute('''CREATE TABLE DimPurchasingStatus (
    PurchasingStatusID int primary key,
    Name varchar(50)
)''')

cursor.execute('''CREATE TABLE DimCustomer (
    CustomerID int primary key,
    PersonName varchar(50),
    StoreName varchar(50),
    TerritoryName varchar(50),
    CountryOrRegion varchar(50),
)''')

cursor.execute('''CREATE TABLE DimSalesStatus (
    SalesStatusID int primary key,
    Name varchar(50)
)''')

cursor.execute('''CREATE TABLE DimOnlineOrderFlag (
    OnlineOrderFlagID int primary key,
    Name varchar(50)
)''')

cursor.commit()


# Create Facts

In [49]:
cursor.execute('''CREATE TABLE FactPurchasingOrders (
    PurchaseOrderID int  not null,
    PurchaseOrderDetailID int  not null,
    ProductID int FOREIGN KEY REFERENCES DimProduct(ProductID),
    StatusID int FOREIGN KEY REFERENCES DimPurchasingStatus(PurchasingStatusID),
    EmployeeID int FOREIGN KEY REFERENCES DimEmployee(EmployeeID),
    VendorID int FOREIGN KEY REFERENCES DimVendor(VendorID),
    ShipMethodID int FOREIGN KEY REFERENCES DimShipMethod(ShipMethodID),
    OrderDate date FOREIGN KEY REFERENCES DimDate(FullDate),
    ShipDate date FOREIGN KEY REFERENCES DimDate(FullDate),
    LineTotal money,
    ReceivedQty decimal(8,2),
    RejectedQty decimal(8,2),
    StockQty decimal(9,2)
    CONSTRAINT PK_PurchasingOrder PRIMARY KEY (PurchaseOrderID,PurchaseOrderDetailID)
)''')
cursor.commit()


In [50]:
cursor.execute('''CREATE TABLE FactSalesOrders (
    SalesOrderID int  not null,
    SalesOrderDetailID int  not null,
    ProductID int FOREIGN KEY REFERENCES DimProduct(ProductID),
    StatusID int FOREIGN KEY REFERENCES DimSalesStatus(SalesStatusID),
    CustomerID int FOREIGN KEY REFERENCES DimCustomer(CustomerID),
    ShipMethodID int FOREIGN KEY REFERENCES DimShipMethod(ShipMethodID),
    OrderDate date FOREIGN KEY REFERENCES DimDate(FullDate),
    DueDate date FOREIGN KEY REFERENCES DimDate(FullDate),
    ShipDate date FOREIGN KEY REFERENCES DimDate(FullDate),
    LineTotal money,
    OrderQty int,
    CONSTRAINT PK_SalesOrder PRIMARY KEY (SalesOrderID,SalesOrderDetailID)

)''')
cursor.commit()
